In [3]:
%config IPCompleter.use_jedi = False

In [11]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu 
import inference
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

In [5]:
model_path = dataiku.Folder("UKTXMoc0").get_path()

In [7]:
model = inference.model_fn(model_path)

In [8]:
image_details_by_image_path = dataiku.Dataset("image_details_by_image_path")

In [12]:
es = Elasticsearch()

In [10]:
def make_feature_vec(row, model):
    data = row['description_concat']
    feature_vec = inference.predict_fn(data, model)
    return feature_vec

In [25]:
sample_row = image_details_by_image_path_df.head(1).to_dict(orient='records')[0]

In [26]:
model_dim = len(make_feature_vec(row, model))

--- Inference time: 0.35622191429138184 seconds ---


In [31]:
# create elasticsearch Index
if es.indices.exists(index='idx_zalando'):
    es.indices.delete(index='idx_zalando')
    
create_query = {
    "mappings": {
        "properties": {
            "zalando_nlu_vector": {
                "type": "dense_vector",
                "dims": model_dim
            }
        }
    }
}

es.indices.create(index='idx_zalando', **create_query)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'idx_zalando'}

In [0]:
def get_embeddings():
    for row in image_details_by_image_path.iter_rows():
        embeddings = make_feature_vec(row, model)
        row['zalando_nlu_vector'] = embeddings
        yield row

In [0]:
bulk(es, get_embeddings())

In [0]:
# Read recipe inputs
image_details_by_image_path = dataiku.Dataset("image_details_by_image_path")
image_details_by_image_path_df = image_details_by_image_path.get_dataframe()


# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a Pandas dataframe
# NB: DSS also supports other kinds of APIs for reading and writing data. Please see doc.

idx_zalando_df = image_details_by_image_path_df # For this sample code, simply copy input to output


# Write recipe outputs
idx_zalando = dataiku.Dataset("idx_zalando")
idx_zalando.write_with_schema(idx_zalando_df)